In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/egutierrezc93/deep_learning_course_rlxmooc/master/teacher/init.py
import init
init.init(force_download=False)
init.get_weblink()

In [2]:
from local.lib.rlxmoocapi import submit, session
import inspect
student = session.Session(init.endpoint).login( course_id=init.course_id, 
                                                session_id="UDEA", 
                                                lab_id="LAB_U3.04" )

In [4]:
import sys
if 'google.colab' in sys.modules:
    print ("setting tensorflow version in colab")
    %tensorflow_version 2.x
    %load_ext tensorboard
import tensorflow as tf
tf.__version__

In [5]:
from time import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from local.lib import mlutils
import pickle, bz2
import numpy as np
%matplotlib inline

# Lab: convolution matrix for convolution and transposed convolutions

Read and understand well how the convolution matrix is built: [Up-sampling with Transposed Convolution](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0)

Observe how a 2d convolution is done with a `tf.layers.conv2d` Keras layer (with linear activation and zero bias)

In [14]:
simg = np.r_[[[4,5,8,7],[1,8,8,8],[3,6,6,4],[6,5,7,8]]].astype(np.float32)
akernel = np.r_[[[1,4,1],[1,4,3],[3,3,1]]]

#simg = np.random.randint(10, size=(10,6))
#akernel = np.random.randint(10, size=(5,5))+1

plt.imshow(simg)

In [15]:
c1 = tf.keras.layers.Conv2D(filters=1, kernel_size=akernel.shape, activation="linear")
c1.build(input_shape=[None, *simg[:,:,None].shape])
w = c1.get_weights()
w[0] = akernel[:,:,None, None]
c1.set_weights(w)

In [16]:
routput = c1(simg[None, :, :, None]).numpy()
print(routput[0,:,:,0])

In [17]:
plt.imshow(routput[0,:,:,0], cmap=plt.cm.Greys_r)


## Task 1: Create the convolution matrix

fill in the following function to create a convolution matrix.

given:

- an input image of dimensions $h\times w$
- a squared filter (kernel) of dimensions $k \times k$
- no padding used

the resulting convolution matrix must have dimensions 

$$(h-k+1)(w-k+1) \times hw$$

suggestions:

- start with the same input image and kernel as in the example in [Up-sampling with Transposed Convolution](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0):
    
       simg = np.r_[[[4,5,8,7],[1,8,8,8],[3,6,6,4],[6,5,7,8]]]
       kernel = np.r_[[[1,4,1],[1,4,3],[3,3,1]]]
       
you should get the following

        array([[1., 4., 1., 0., 1., 4., 3., 0., 3., 3., 1., 0., 0., 0., 0., 0.],
               [0., 1., 4., 1., 0., 1., 4., 3., 0., 3., 3., 1., 0., 0., 0., 0.],
               [0., 0., 0., 0., 1., 4., 1., 0., 1., 4., 3., 0., 3., 3., 1., 0.],
               [0., 0., 0., 0., 0., 1., 4., 1., 0., 1., 4., 3., 0., 3., 3., 1.]])
       
- test with random imgs and kernels (as in the code above) with increasing dimensions and images with different width and height

In [ ]:
def get_convmatrix(input_shape, kernel):
    # IMPORT HERE ANY LIBRARIES YOU NEED
    import numpy as np
    
    dx, dy = np.r_[input_shape[0] - kernel.shape[0]+1, input_shape[1] - kernel.shape[1]+1]
    cm = np.zeros((np.product(input_shape), dx*dy)).T
    cx,cy=0,0
    for cx in range(dx):
        for cy in range(dy):
            for i in range(kernel.shape[1]):
                .... # your code here
    return cm

In [21]:
cm = get_convmatrix(simg.shape, akernel)
cm.shape

In [22]:
cm

**Submit your solution**

In [27]:
student.submit_task(namespace=globals(), course_id=init.course_id, lab_id='LAB_U4.10', task_id='task_01')

## Task 2: use the convolution matrix above to make a 2d convolution

you must get the same 2x2 result matrix as above

In [29]:
def conv2d(img,kernel):
    # IMPORT HERE ANY LIBRARIES YOU NEED
    
    ... # your code here
    moutput = ... # your code here
    return moutput

In [31]:
moutput = conv2d(simg, akernel)
moutput

the following must come out `True`

In [32]:
np.allclose(routput, moutput.reshape(routput.shape))

**Submit your solution**

In [37]:
student.submit_task(namespace=globals(), course_id=init.course_id, lab_id='LAB_U4.10', task_id='task_02')

## Task 3: make a transposed 2d convolution using `tf.keras.layers.Conv2DTranspose`

- Convolution took a 4x4 image into a 2x2 activation map
- Transposed convolution will take a 2x2 input image into a 4x4 activation map

`small_img` will be our 2x2 input image for the transposed convolution.

for `small_img` =

        array([[6., 5.],
               [3., 5.]], dtype=float32)

you must get the following result

        array([[ 6., 29., 26.,  5.],
               [ 9., 46., 61., 20.],
               [21., 50., 50., 20.],
               [ 9., 24., 18.,  5.]], dtype=float32)


In [38]:
small_img = np.array([[6., 5.],[3., 5.]]).astype(np.float32)
small_img

In [ ]:
def tf_conv2d_transpose(small_img, kernel):
    # IMPORT HERE ANY LIBRARIES YOU NEED
    import tensorflow as tf
    
    ct = tf.keras.layers.Conv2DTranspose(... # your code here)
    # your code here
    return ct(# your code here).numpy()

In [40]:
toutput = tf_conv2d_transpose(small_img, akernel)
toutput[0,:,:,0]

**Submit your solution**

In [45]:
student.submit_task(namespace=globals(), course_id=init.course_id, lab_id='LAB_U4.10', task_id='task_03')

## Task 4: Make a transposed 2d convolution using the convolution matrix above

your result must be exactly the same as the previous task

In [ ]:
def conv2d_transpose(img, kernel):
    # IMPORT HERE ANY LIBRARIES YOU NEED
    
    # your code here
    return # your code here

In [47]:
moutput = conv2d_transpose(small_img, akernel)
moutput

In [48]:
np.allclose(toutput[0,:,:,0], moutput )

**Submit your solution**

In [53]:
student.submit_task(namespace=globals(), course_id=init.course_id, lab_id='LAB_U4.10', task_id='task_04')